In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import os


In [ ]:
def constructGraph_EM(mod_num):
    
    nodes = pd.read_csv("./EM_communities/averageSubComm%d_drosophilia.csv"%mod_num).Node.to_list()
    
    # EM network for Module 
    
    df = pd.read_csv("traced-total-connections.csv")
    nodeA_list = df.bodyId_pre.tolist()
    nodeB_list = df.bodyId_post.tolist()
    weight_list = df.weight.tolist()
    
    # creating an edge list from adjacency matrix
    edge_list=[]
    for l,m,w in zip(nodeA_list, nodeB_list, weight_list):
        if (l in nodes) and (m in nodes):
            edge_list.append( (l,m,{"weight":w}))
    
    G_EM = nx.DiGraph()
    
    # Add all the nodes to the graph
    G_EM.add_nodes_from(nodes)
    # Add all the edges to the graph
    G_EM.add_edges_from(edge_list)
    return G_EM
    

In [ ]:
def constructGraph_model(mod_num, pr):

    
    path = "./janelia_Comm%d/"%mod_num
    nodes = pd.read_csv("./EM_communities/averageSubComm%d_drosophilia.csv"%mod_num).Node.to_list()
    
    df_network = pd.read_csv(path+"network_%.1fum.txt"%pr, sep="\t", header=None)
    df_network = df_network.rename(columns={0: "bodyId_pre", 1: "bodyId_post", 2:"weight"})
    
    nodeA_list = df_network.bodyId_pre.tolist()
    nodeB_list = df_network.bodyId_post.tolist()
    weight_list = df_network.weight.tolist()
    
    # creating an edge list from adjacency matrix
    edge_list=[]
    for l,m,w in zip(nodeA_list, nodeB_list, weight_list):
        if (l in nodes) and (m in nodes):
            edge_list.append( (l,m,{"weight":w}))
        
    G_strah = nx.DiGraph()
    
    # Add all the nodes to the graph
    G_strah.add_nodes_from(nodes)
    # Add all the edges to the graph
    G_strah.add_edges_from(edge_list)
    return G_strah

In [ ]:
def plot_constbin(degree):
    degree = np.array(degree) # if list convert to np array
    deg = degree[degree>0] # don't consider values=0
    #binsize=75
    #nbins = int((max(deg)-min(deg))/binsize)
    edges= np.histogram_bin_edges(deg,bins='auto')#=nbins)# bins="auto"
    pk = np.array(plt.hist(deg,bins=edges,density=True)[0])

    plt.close()

    edges2 = list(edges[1:])+[0]

    mid_point = (np.array([a+b for a,b in zip(edges,edges2)])/2)[:-1]
    return mid_point, pk
    

In [ ]:
image_path = "./comm_degDistr_images/"
if not os.path.isdir(image_path):
    os.mkdir(image_path)
image_path = "./comm_degDistr_images/indegree/"
if not os.path.isdir(image_path):
    os.mkdir(image_path)
image_path = "./comm_degDistr_images/outdegree/"
if not os.path.isdir(image_path):
    os.mkdir(image_path)

In [ ]:
def compare_indegree(mod_num, pr):
    G_EM = constructGraph_EM(mod_num)
    G_strah = constructGraph_model(mod_num, pr)

    EM_in_deg = list(dict(G_EM.in_degree(weight='weight')).values())
    #EM_out_deg = list(dict(G_EM.out_degree(weight='weight')).values())
    
    strah_in_deg = list(dict(G_strah.in_degree(weight='weight')).values())
    #strah_in_deg = list(dict(G_strah.out_degree(weight='weight')).values())

    
    mid_point1, pk1 = plot_constbin(EM_in_deg)

    mid_point2, pk2 = plot_constbin(strah_in_deg)
    
    
    plt.loglog(mid_point1, pk1,'o', alpha=0.7, color='#d62728', label='EM In-degree')
    
    plt.loglog(mid_point2, pk2,'o', alpha=0.7,color='#9467bd', label='Model In-degree')
    #plt.axvline(3000,ls="--", linewidth=2.5,color="k" )
    
    plt.tick_params(direction='in',bottom=True,top=True,left=True,right=True)
    plt.grid(linestyle='--',alpha=0.5,which='both')
    plt.xlabel("In-degree",size=15)
    plt.ylabel("Degree distribution",size=15)
    plt.title(r"Module %d, PR %.1f$\mu$m"%(mod_num+1, pr))
    
    plt.legend(loc='best')#, prop={'size':15})
    
    plt.rcParams['font.size']=15
    
    image_path = "./comm_degDistr_images/indegree/"

    plt.savefig(image_path+"mod%d_pr%.1f.pdf"%((mod_num+1), pr),
                facecolor="white", bbox_inches="tight", dpi=600)

In [ ]:
def compare_outdegree(mod_num, pr):
    G_EM = constructGraph_EM(mod_num)
    G_strah = constructGraph_model(mod_num, pr)

    EM_out_deg = list(dict(G_EM.out_degree(weight='weight')).values())
    
    strah_out_deg = list(dict(G_strah.out_degree(weight='weight')).values())
    
    mid_point1, pk1 = plot_constbin(EM_out_deg)

    mid_point2, pk2 = plot_constbin(strah_out_deg)
    
    plt.loglog(mid_point1, pk1,'o', alpha=0.7, color='#d62728', label='EM Out-degree')
    
    plt.loglog(mid_point2, pk2,'o', alpha=0.7,color='#9467bd', label='Model Out-degree')
    #plt.axvline(3000,ls="--", linewidth=2.5,color="k" )
    
    plt.tick_params(direction='in',bottom=True,top=True,left=True,right=True)
    plt.grid(linestyle='--',alpha=0.5,which='both')
    plt.xlabel("Out-degree",size=15)
    plt.ylabel("Degree distribution",size=15)
    plt.title(r"Module %d, PR %.1f$\mu$m"%(mod_num+1, pr))
    
    plt.legend(loc='best')#, prop={'size':15})
    
    plt.rcParams['font.size']=15
    
    image_path = "./comm_degDistr_images/outdegree/"

    plt.savefig(image_path+"mod%d_pr%.1f.pdf"%((mod_num+1), pr),
                facecolor="white", bbox_inches="tight", dpi=600)

In [ ]:
for mod_num in range(8):
    compare_indegree(mod_num, 1.0)
    compare_outdegree(mod_num, 1.0)
    
    compare_indegree(mod_num, 5.0)
    compare_outdegree(mod_num, 5.0)
